In [ ]:
import gym
import numpy as np
import random
import math
from time import sleep


## Initialize the "Cart-Pole" environment
env = gym.make('CartPole-v0')

## Defining the environment related constants

# Number of discrete states (bucket) per state dimension
NUM_BUCKETS = (1, 1, 6, 3)  # (x, x', theta, theta')
# Number of discrete actions
NUM_ACTIONS = env.action_space.n # (left, right)
# Bounds for each discrete state
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]
# Index of the action
ACTION_INDEX = len(NUM_BUCKETS)

## Creating a Q-Table for each state-action pair
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

## Learning related constants
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1

## Defining the simulation related constants
NUM_EPISODES = 1000
MAX_T = 250
STREAK_TO_END = 120
SOLVED_T = 199
DEBUG_MODE = True

def simulate():

    ## Instantiating the learning related parameters
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    discount_factor = 0.99  # since the world is unchanging

    num_streaks = 0

    for episode in range(NUM_EPISODES):

        # Reset the environment
        obv = env.reset()

        # the initial state
        state_0 = state_to_bucket(obv)

        for t in range(MAX_T):
            env.render()

            # Select an action
            action = select_action(state_0, explore_rate)

            # Execute the action
            obv, reward, done, _ = env.step(action)

            # Observe the result
            state = state_to_bucket(obv)

            # Update the Q based on the result
            best_q = np.amax(q_table[state])
            q_table[state_0 + (action,)] += learning_rate*(reward + discount_factor*(best_q) - q_table[state_0 + (action,)])

            # Setting up for the next iteration
            state_0 = state

            # Print data
            if (DEBUG_MODE):
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)

                print("")

            if done:
                print("Episode %d finished after %f time steps" % (episode, t))
                if (t >= SOLVED_T):
                    num_streaks += 1
                else:
                    num_streaks = 0
                break

            #sleep(0.25)

        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > STREAK_TO_END:
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)


def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = np.argmax(q_table[state])
    return action


def get_explore_rate(t):
    return max(MIN_EXPLORE_RATE, min(1, 1.0 - math.log10((t+1)/25)))

def get_learning_rate(t):
    return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))

def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

if __name__ == "__main__":
    simulate()


Episode = 0
t = 0
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 1
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 0.500000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 2
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 0.747500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 3
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 1.243763
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 4
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 1.365662
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 5
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 0
t = 6
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 0.000000
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episo



Episode = 3
t = 6
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 4.705902
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 7
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 5.090894
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 8
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 5.565440
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 9
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 5.607844
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 10
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 6.079805
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 11
Action: 1
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 0.997500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 3
t = 12
Action: 0
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 0.997500
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


E


Episode = 7
t = 3
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 7.745825
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 4
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.070908
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 5
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.368012
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 6
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 8.677620
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 7
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 7.560150
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 8
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 7.560150
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 7
t = 9
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 8.022349
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episo


Episode = 9
t = 16
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 11.795986
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 17
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 11.960758
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 18
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 12.400954
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 19
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 12.536465
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 20
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 12.906028
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 21
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 13.156716
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 9
t = 22
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 13.465588
Explore rate: 1.000000
Learning rate: 0.500000
Stre


Episode = 11
t = 18
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 12.336208
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 19
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 13.524484
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 20
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 13.524484
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 21
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 13.956862
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 22
Action: 1
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 2.166061
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 23
Action: 1
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 2.166061
Explore rate: 1.000000
Learning rate: 0.500000
Streaks: 0


Episode = 11
t = 24
Action: 1
State: (0, 0, 1, 0)
Reward: 1.000000
Best Q: 2.501295
Explore rate: 1.000000
Learning rate: 0.500000


...


Episode = 166
t = 136
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 85.941183
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 137
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 85.941183
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 138
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 85.966184
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 139
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 85.885676
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 140
Action: 0
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 85.997486
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 141
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 85.707481
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 142
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 85.707481
Explore rate: 0.177832
Learnin

Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 197
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 86.005097
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 198
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.254039
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8


Episode = 166
t = 199
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.254039
Explore rate: 0.177832
Learning rate: 0.177832
Streaks: 8

Episode 166 finished after 199.000000 time steps

Episode = 167
t = 0
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 86.226441
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 1
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 86.145553
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 2
Action: 1
State: (0, 0, 3, 0)
Reward: 1.000000
Best Q: 86.145553
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t


Episode = 167
t = 57
Action: 1
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 86.193991
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 58
Action: 0
State: (0, 0, 2, 0)
Reward: 1.000000
Best Q: 86.193991
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 59
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.452036
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 60
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.452036
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 61
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.475775
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 62
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.430114
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 63
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.430114
Explore rate: 0.175224
Learning rate:


Episode = 167
t = 118
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.622448
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 119
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.483650
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 120
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.641144
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 121
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.534654
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 122
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 86.692293
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 123
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 86.692293
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 124
Action: 1
State: (0, 0, 3, 1)
Reward: 1.000000
Best Q: 86.715611
Explore rate: 0.175224
Learnin


Episode = 167
t = 178
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.930965
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 179
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.930965
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 180
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.953865
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 181
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.874412
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 182
Action: 1
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.981796
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 183
Action: 0
State: (0, 0, 2, 1)
Reward: 1.000000
Best Q: 86.981796
Explore rate: 0.175224
Learning rate: 0.175224
Streaks: 9


Episode = 167
t = 184
Action: 0
State: (0, 0, 2, 2)
Reward: 1.000000
Best Q: 86.916039
Explore rate: 0.175224
Learnin